# How to work with data in the cloud?

The [MAST data archive](https://archive.stsci.edu/) provides a copy of several large NASA data sets via Amazon Web Services (AWS) as part of the [MAST AWS Public Datasets](https://registry.opendata.aws/collab/stsci/) program.

This service is of interest to TIKE users, because the TIKE platform is hosted by AWS and therefore provides fast access to these data sets. The data are free to access and download: they do not require you to have an AWS account or use AWS credentials.

Below we demonstrate a few ways to access the data.


## Which data sets are available in the cloud?

At the time of writing, selected data sets from Hubble, Kepler, K2, and TESS are available via the AWS S3 cloud file storage service.  You can read more details about this service via the following MAST blog posts:

* [Making HST Public Data Available on AWS](https://mast-labs.stsci.io/2018/06/hst-public-data-on-aws)
* [TESS data available on AWS](https://mast-labs.stsci.io/2018/12/tess-data-available-on-aws)
* [Kepler Prime Mission Data Available on AWS](https://mast-labs.stsci.io/2019/10/kepler-data-available-on-aws)

## How can I access the cloud-hosted data?

The data can be accessed using various client libraries, including astroquery, lightkurve, aws-cli, boto3, s3fs, and others. Below we demonstrate a few basic ways to use these tools.

### 1. Using `astroquery`

The [astroquery](https://astroquery.readthedocs.io) package provides a special function, [enable_cloud_dataset()](https://astroquery.readthedocs.io/en/latest/api/astroquery.mast.ObservationsClass.html#astroquery.mast.ObservationsClass.enable_cloud_dataset), which enables users to download data products from AWS S3 rather than MAST's own servers.  Astroquery is pre-installed on TIKE and should work out of the box.

For example, we can query and download the TESS Sector 11 pixel file for Proxima Centauri as follows:

In [ ]:
# Query MAST for TESS Sector 11 observations of Proxima Cen
from astroquery.mast import Observations
obs = Observations.query_object("Proxima Cen", radius="0s")
want = (obs['obs_collection'] == "TESS") & (obs['sequence_number'] == 11) & (obs['dataproduct_type'] == 'timeseries')

# Pick which products we want to retrieve
data_prod = Observations.get_product_list(obs[want])
filt_prod = Observations.filter_products(data_prod, description="Target pixel files")

# Download the product from AWS S3
Observations.enable_cloud_dataset(provider='AWS')
manifest = Observations.download_products(filt_prod, cloud_only=True)

You can read more about this feature in the [astroquery documentation](https://astroquery.readthedocs.io/en/latest/mast/mast.html?highlight=cloud#cloud-data-access).

### 2. Using `lightkurve`

The [Lightkurve](https://docs.lightkurve.org) package provides search functions which use Astroquery to search and retrieve specific TESS and Kepler data products in a user-friendly way.  For example, we can download the same Proxima Centauri pixel file we obtained above as follows:

In [ ]:
# Important: ensure files are retrieved from AWS 
from astroquery.mast import Observations
Observations.enable_cloud_dataset(provider='AWS')

import lightkurve as lk
search = lk.search_targetpixelfile("Proxima Cen", sector=11, mission="TESS")
pixelfile = search[0].download(download_dir=".")

Again, the call to Astroquery's `enable_cloud_dataset()` function is key to ensure that data is downloaded from AWS rather than MAST. This works because Lightkurve uses Astroquery behind the scenes.

### 3. Using the AWS command-line tool

S3-hosted data can also be accessed in a more direct way using the standard tools available to work with AWS. For example, a useful tool to explore data in S3 is the [AWS command-line interface](https://docs.aws.amazon.com/cli/). This tool is pre-installed on the TIKE platform and can be executed from a Terminal window (<span style="font-variant:small-caps;">file › new › terminal</span>) or from a Jupyter notebook (by prefixing it with the `!` character).

For example, we can list the contents of the TESS data held in the public MAST S3 bucket as follows: 

In [ ]:
!aws s3 ls s3://stpubdata/tess/public/ --no-sign-request

The data in this bucket follow a specific directory structure and file name convention ([documented here](https://archive.stsci.edu/missions-and-data/tess/data-products)). For example, the TESS Sector 11 data products for Proxima Centauri (also known as TIC 388857263) are available at the following location:

In [ ]:
!aws s3 ls s3://stpubdata/tess/public/tid/s0011/0000/0003/8885/7263/ --no-sign-request

We can copy files from this location using the `aws s3 cp` command.  For example, we can obtain the Target Pixel File (`*tp.fits`) for Proxima Centauri as follows:

In [ ]:
!aws s3 cp s3://stpubdata/tess/public/tid/s0011/0000/0003/8885/7263/tess2019112060037-s0011-0000000388857263-0143-s_tp.fits . --no-sign-request

See the AWS [command line interface documentation](https://docs.aws.amazon.com/cli/latest/reference/s3/) to explore additional features.

### 4. Using `s3fs`

The [s3fs](https://s3fs.readthedocs.io/en/latest/) package provides a Pythonic file interface to AWS S3.  This enables us to execute the commands demonstrated above from Python rather than the command line. For example:

In [ ]:
# Initialize the tool
import s3fs
fs = s3fs.S3FileSystem(anon=True)

# List the directory containing Proxima Cen data
fs.ls('s3://stpubdata/tess/public/tid/s0011/0000/0003/8885/7263/', refresh=True)

# Transfer the Target Pixel File for Proxima Cen from S3
f = fs.get('s3://stpubdata/tess/public/tid/s0011/0000/0003/8885/7263/tess2019112060037-s0011-0000000388857263-0143-s_tp.fits', 'proxima_cen_tp.fits')

## Further reading

Additional examples are available here:
- [Using Cloud-Hosted Data](../code/cloud_astroquery.ipynb)
- [Reading Public Kepler Data from S3](../code/s3buckets_boto3.ipynb)
- Creating cutouts from TESS FFI data hosted in AWS (content in progress)
- How to read and write data to your own S3 bucket (content in progress)